In [ ]:
!pip install kaggle-environments -U > /dev/null 2>&1
!cp -r ../input/lux-ai-2021/* .

In [ ]:
import numpy as np
import json
from pathlib import Path
import os
import random
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
model = torch.jit.load("../input/models-lux/model_dr.pth")
model.eval()
traced = torch.jit.trace(model.cpu(), torch.rand(1, 20, 32, 32))
traced.save('model_dr.pth')

In [ ]:
model = torch.jit.load("../input/models-lux/model_toad.pth")
model.eval()
traced = torch.jit.trace(model.cpu(), torch.rand(1, 20, 32, 32))
traced.save('model_toad.pth')

In [ ]:
model = torch.jit.load("../input/models-lux/model_rl.pth")
model.eval()
traced = torch.jit.trace(model.cpu(), torch.rand(1, 20, 32, 32))
traced.save('model_rl.pth')

# Submission

In [ ]:
%%writefile agent.py
import os
import numpy as np
import torch
from lux.game import Game
from collections import Counter
from torch import nn


path = '/kaggle_simulations/agent' if os.path.exists('/kaggle_simulations') else '.'

model = torch.jit.load(f"{path}/model_dr.pth")
model.eval()

model2 = torch.jit.load(f"{path}/model_toad.pth")
model2.eval()

model3 = torch.jit.load(f"{path}/model_rl.pth")
model3.eval()


def make_input(obs, unit_id):
    width, height = obs['width'], obs['height']
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    
    b = np.zeros((20, 32, 32), dtype=np.float32)
    
    for update in obs['updates']:
        strs = update.split(' ')
        input_identifier = strs[0]
        
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3]:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]   
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs['player']) % 2 * 2
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[17, :] = obs['step'] % 40 / 40
    # Turns
    b[18, :] = obs['step'] / 360
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b

game_state = None
def get_game_state(observation):
    global game_state
    
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation["player"]
    else:
        game_state._update(observation["updates"])
    return game_state


def in_city(pos):    
    try:
        city = game_state.map.get_cell_by_pos(pos).citytile
        return city is not None and city.team == game_state.id
    except:
        return False


def call_func(obj, method, args=[]):
    return getattr(obj, method)(*args)


unit_actions = [('move', 'n'), ('move', 's'), ('move', 'w'), ('move', 'e'), ('build_city',)]

def get_action(policy, unit, dest):

    for label in np.argsort(policy)[::-1]:
        act = unit_actions[label]
        pos = unit.pos.translate(act[-1], 1) or unit.pos
        if pos not in dest or in_city(pos):
            return call_func(unit, *act), pos 
            
    return unit.move('c'), unit.pos

def get_action_unet(policy, unit, dest, shift):
    logits = nn.Softmax(policy[:, unit.pos.x + shift, unit.pos.y + shift] )
    action = unet_unit_actions[ np.argmax( policy[:, unit.pos.x + shift, unit.pos.y + shift] )]
    pos = unit.pos.translate(action[-1], 1) or unit.pos
    if pos not in dest or in_city(pos):
        return call_func(unit, *action), pos
    
    return unit.move('c'), unit.pos 


def agent(observation, configuration):
    global game_state
    
    game_state = get_game_state(observation)    
    player = game_state.players[observation.player]
    actions = []
    
        
    # City Actions
    unit_count = len(player.units)
    for city in player.cities.values():
        for city_tile in city.citytiles:
            if city_tile.can_act():
                if unit_count < player.city_tile_count: 
                    actions.append(city_tile.build_worker())
                    unit_count += 1
                elif not player.researched_uranium():
                    actions.append(city_tile.research())
                    player.research_points += 1
    
    dest = []

    for unit in player.units:
        if unit.can_act() and (game_state.turn % 40 < 30 or not in_city(unit.pos)):
            state = make_input(observation, unit.id)
            with torch.no_grad():
                p = model(torch.from_numpy(state).unsqueeze(0))
                p2 = model2(torch.from_numpy(state).unsqueeze(0))
                p3 = model3(torch.from_numpy(state).unsqueeze(0))
                
            
            policy = p.squeeze(0).numpy() 
            policy2 = p2.squeeze(0).numpy()
            policy3 = p3.squeeze(0).numpy()
            
            softmax = nn.Softmax(dim=0)
            logits1 = softmax(torch.from_numpy(policy))
            logits2 = softmax(torch.from_numpy(policy2))
            logits3 = softmax(torch.from_numpy(policy3))
            
            ensemble_logits = np.array( logits1 * 0.6 + logits2 * 0.2 + logits3 * 0.2)

            action, pos = get_action(ensemble_logits, unit, dest)
            
        
            #action2, pos = get_action(policy2, unit, dest)
            #action3, pos = get_action(policy3, unit, dest)
            #action4, pos = get_action_unet(policy4, unit, dest, shift)
            
            #action = Counter([action,action,action,
            #                  action2,action2,]).most_common(1)[0][0]
            actions.append(action)
            dest.append(pos)

    return actions

In [ ]:
np.argsort( np.array([1,2,44,7,5])[::-1] )

In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"width": 24, "height": 24, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run(['agent.py', 'agent.py'])
env.render(mode="ipython", width=1200, height=800)

In [ ]:
!tar -czf submission.tar.gz *